In [1]:
# %load ../scripts/post-processing.py
"""Process the patch files generated by extractPatches.py and prepare
them for analysis by Kmeans and diffusion-maps"""

from glob import glob
import pickle as pk
from os import system
import numpy as np
from os.path import isdir
#from minio import Minio

import pickle as pk
import numpy as np
from glob import glob

from os import mkdir

In [2]:
# %load /home/ubuntu/shapeology_code/scripts/lib/utils.py
import psutil
from os import system
from subprocess import Popen,PIPE
from time import time

import numpy as np
from numpy import sqrt, mean
#utils that do not require plotting
from subprocess import Popen,PIPE
from os import system
from os.path import isfile,getmtime

import cv2

#import matplotlib.pyplot as plt
#from astropy.convolution import Gaussian2DKernel,convolve

import yaml

class configuration():
    def __init__(self,yamlFile):
        self.D=yaml.load(open(yamlFile,'r'))

    def getParams(self):
        return self.D

def run(command):
    print('cmd=',command)
    system(command)
    
def runPipe(command):
    print('runPipe cmd=',command)
    p=Popen(command.split(),stdout=PIPE,stderr=PIPE)
    L=p.communicate()
    stdout=L[0].decode("utf-8").split('\n')
    stderr=L[1].decode("utf-8").split('\n')
    return stdout,stderr

time_log=[]
def clock(message):
    print('%8.1f \t%s'%(time(),message))
    time_log.append((time(),message))

def printClock():
    t=time_log[0][0]
    for i in range(1,len(time_log)):
        print('%8.1f \t%s'%(time_log[i][0]-t,time_log[i][1]))
        t=time_log[i][0]

def list_s3_files(stack_directory):
    stdout,stderr=runPipe("aws s3 ls %s/ "%(stack_directory))
    filenames=[]
    for line in stdout:
        parts=line.strip().split()
        if len(parts)!=4:
            continue
        filenames.append(parts[-1])
    return filenames

def read_files(s3_dir,_delete=False,data_dir='/dev/shm/data/'):
    s3files=list_s3_files(s3_dir)
    for filename in s3files:
        if not isfile(data_dir+'/'+filename):
            run('aws s3 cp %s/%s %s'%(s3_dir,filename,data_dir))
        D=fromfile(data_dir+'/'+filename,dtype=np.float16)
        pics=D.reshape([-1,_size,_size])
        if _delete:
            run('rm %s/%s'%(data_dir,filename))
        yield pics

def data_stream(s3_dir='s3://mousebraindata-open/MD657/permuted'):
    for pics in read_files(s3_dir):
        j=0
        for i in range(pics.shape[0]):
            if j%1000==0:
                print('\r examples read=%10d'%j,end='')
            j+=1    
            yield pics[i,:,:]


def Last_Modified(file_name):
    try:
        mtime = getmtime(file_name)
    except OSError:
        mtime = 0
    return(mtime)
            
def calc_err(pic,gaussian = None):
    if gaussian is None:
        gaussian=Gaussian2DKernel(1,x_size=7,y_size=7)
    factor=np.sum(gaussian)
    P=convolve(pic,gaussian)/factor
    #except:
    #    print('err in calc_err/convolve',pic.shape,gaussian.shape,factor)
    #    P=pic
    error=sqrt(mean(abs(pic-P)))
    sub=P[::2,::2]
    return error,sub

# def plot_patches(fig,data,h=15,w=15,_titles=[]):
#     for i in range(h*w):
#         if i>=data.shape[0]:
#             break
#         ax=fig.add_subplot(h,w,i+1);
#         pic=np.array(data[i,:,:],dtype=np.float32)

#         subfig=ax.imshow(pic,cmap='gray')
#         if(len(_titles)>i):
#             plt.title(_titles[i])
#         subfig.axes.get_xaxis().set_visible(False)
#         subfig.axes.get_yaxis().set_visible(False)

def pack_pics(Reps):
    size=Reps[0].shape[0]
    _len=len(Reps)
    Reps_mat=np.zeros([_len,size,size])
    for i in range(_len):
        Reps_mat[i,:,:]=Reps[i]
    return Reps_mat

def dist2(a,b):
    diff=(a-b)**2
    return np.sum(diff.flatten())

# def dist_hist(data):
#     D=[]
#     for i in range(1,data.shape[0]):
#         D.append(dist2(data[i,:,:],data[i-1,:,:]))
#         if i%1000==0:
#             print('\r',i,end='')
#     hist(D,bins=100);

# def find_threshold(image,percentile):
#     """find the threshold at the given percentile

#     :param image: grey-level image
#     :param percentile: threshold is chosen so that 
#                        percentile of the pixels are lower than it
#     :returns: threshold
#     :rtype: float

#     """
#     V=sorted(image.flatten())
#     l=len(V)
#     thr=V[int(l*percentile)] 
#     return thr

def mark_contours(D,tile):

    image = np.array(tile,dtype=np.uint8)
    kernel = np.ones((3,3),np.uint8)
    boundary=np.zeros(image.shape,np.uint8)
    repress=boundary.copy()
    #print('shape of boundary=',boundary.shape)
    _shape=D[0]['mask'].shape
    left=int(_shape[0]/2)
    right=_shape[0]-left
    left,right

    for R in D:
        #compute contour
        color=np.array([0,0,0],dtype=np.uint8)
        color[R['j'] % 2]=255
        mask=np.array(R['mask']*1,dtype=np.uint8)
        dilated = cv2.dilate(mask,kernel,iterations = 1)
        contour = dilated-mask
        #mark contour in ln image coordinates
        coor=[R['Y'],R['X']]

        boundary[coor[0]-left:coor[0]+right,coor[1]-left:coor[1]+right]\
        +=np.multiply.outer(contour,color)
        repress[coor[0]-left:coor[0]+right,coor[1]-left:coor[1]+right]\
        +=np.multiply.outer(contour,np.array([1,1,1],dtype=np.uint8))

    combined=image.copy()
    combined[repress==1]=255
    # combined +=boundary

    return combined


In [3]:
import sys
sys.path.append('../scripts/')
from lib.utils import *
from lib.permute import permutator
from Controller import get_file_table

In [4]:
# %load /home/ubuntu/shapeology_code/scripts/lib/permute.py
import numpy as np
from numpy.random import permutation
from os import mkdir, remove
from glob import glob

class permutator:
    """ A class that creates a random permutation of equal-size bytearrays"""
    N=0 #permutor index (used to differentiate files) 
    def __init__(self,element,K=100,temp_root='/tmp'):  #number of files used for randomization
        assert type(element) == bytearray
        self.nbytes=len(element)
        print(self.nbytes,'bytes per element')
        self.tmpDir=temp_root+'/permuted-%d'%permutator.N
        self.K=K
        permutator.N+=1
        try:
            mkdir(self.tmpDir)
            print('made dir ',self.tmpDir)
        except:
            print(self.tmpDir,'already exists, removing contents')
            for file in glob(self.tmpDir+'/*'):
                remove(file)
        
        self.fp=[]
        for i in range(K):
            self.fp.append(open(self.tmpDir+'/bucket-'+str(i)+'.bin','bw'))

    def push(self,element):
        assert type(element) == bytearray
        if len(element) != self.nbytes:
            throw('Inconsistent element size: init=%d, current=%d'%(self.nbytes,element.nbytes))
        j=np.random.randint(self.K)
        #print(type(element),len(Bytes))
        self.fp[j].write(element)
    
    def combine(self,outfilename):
        # read and permute each file
        for i in range(self.K):
            self.fp[i].close()
            self.fp[i]=open(self.tmpDir+'/bucket-'+str(i)+'.bin','br')
        outfile=open(outfilename,'bw')
        for filename in self.fp:
            D=filename.read()
            #print('buffer length',len(D))
            D=np.frombuffer(D,dtype=np.byte)
            #print(D.shape,type(D[0]))
            elements=D.reshape([-1,self.nbytes])
            #print(elements.shape)

            L=elements.shape[0]
            _order=permutation(L)
            permuted_elements=elements[_order,:]
            permuted_elements.tofile(outfile)
            error=elements.nbytes % self.nbytes
            if error!=0:
                throw('incorrect file size:',filename,'shape=',elements.nbytes,'self.nbytes=',self.nbytes,'error=',error)


In [5]:
class Sorter:

    def __init__(self,temp_root='/tmp'):
        self.temp_root=temp_root
        self.Permutators={}

    def sort_file(self,pkl_file):
        #print(pkl_file)
        V=pk.load(open(pkl_file,'rb'))

        for e in V:
            size = e['padded_size']
            tile=e['padded_patch']
            if tile is None:
                continue
            asbytes = bytearray(tile.tobytes())
            #print('size:',size,len(asbytes))

            if not size in self.Permutators:
                self.Permutators[size] = permutator(asbytes,temp_root=self.temp_root)

            self.Permutators[size].push(asbytes)

    def close(self,stem='permuted-'):
        for size in self.Permutators.keys():
            Permutator=self.Permutators[size]
            Permutator.combine('%s-%d'%(stem,size))

#if __name__=='__main__':
    

In [20]:
temp_root='/dev/shm/tmp/'
!mkdir $temp_root
sorter=Sorter(temp_root=temp_root)

mkdir: cannot create directory ‘/dev/shm/tmp/’: File exists


In [21]:
unpack_dir='/dev/shm/data/unpack'
%cd $unpack_dir

s3_dir = 's3://mousebraindata-open/MD657'
AWS_list=list_s3_files(s3_dir)

/dev/shm/data/unpack
runPipe cmd= aws s3 ls s3://mousebraindata-open/MD657/ 


In [22]:
for date,time,size,name in AWS_list:
    if int(size)>1000 and 'extract' in name:
        print(date,time,size,name)
        run('aws s3 cp {0}/{1} {2}/{1}'.format(s3_dir,name,'.'))
        !rm -rf pickles
        run('tar xzf '+name)
        !mv dev/shm/data/tiles/pickles pickles
        !rm -rf dev
        for pickle in glob('pickles/*.pkl'):
            print('\r',pickle,end='')
            sorter.sort_file(pickle)
        !rm $name
        


2019-02-02 22:38:43 61542995 MD657-N1-2017.02.27-15.52.27_MD657_1_0001_lossless_extracted.tgz
cmd= aws s3 cp s3://mousebraindata-open/MD657/MD657-N1-2017.02.27-15.52.27_MD657_1_0001_lossless_extracted.tgz ./MD657-N1-2017.02.27-15.52.27_MD657_1_0001_lossless_extracted.tgz
cmd= tar xzf MD657-N1-2017.02.27-15.52.27_MD657_1_0001_lossless_extracted.tgz
 pickles/tiles_91.aws.pkl450 bytes per element
made dir  /dev/shm/tmp//permuted-3
20402 bytes per element
made dir  /dev/shm/tmp//permuted-4
126002 bytes per element
made dir  /dev/shm/tmp//permuted-5
 pickles/tiles_106.aws.pkl2019-02-03 09:51:14 70225559 MD657-N1-2017.02.27-15.52.27_MD657_2_0002_lossless_extracted.tgz
cmd= aws s3 cp s3://mousebraindata-open/MD657/MD657-N1-2017.02.27-15.52.27_MD657_2_0002_lossless_extracted.tgz ./MD657-N1-2017.02.27-15.52.27_MD657_2_0002_lossless_extracted.tgz
cmd= tar xzf MD657-N1-2017.02.27-15.52.27_MD657_2_0002_lossless_extracted.tgz
 pickles/tiles_00.aws.pkll2019-02-02 23:17:04 80348756 MD657-N1-2017.02.2

OSError: [Errno 28] No space left on device

In [58]:
!rm -rf /dev/shm/tmp/
!du -sh /dev/tmp/*

2.0M	/dev/tmp/permuted-0
313M	/dev/tmp/permuted-1
28M	/dev/tmp/permuted-2
141M	/dev/tmp/permuted-3
24G	/dev/tmp/permuted-4
2.7G	/dev/tmp/permuted-5
0	/dev/tmp/tmp


In [59]:
!df

Filesystem     1K-blocks     Used Available Use% Mounted on
udev            32226128 27660456   4565672  86% /dev
tmpfs            6446908     8684   6438224   1% /run
/dev/nvme0n1p1  32461660  4364520  28080756  14% /
tmpfs           32234524 31884432    350092  99% /dev/shm
tmpfs               5120        0      5120   0% /run/lock
tmpfs           32234524        0  32234524   0% /sys/fs/cgroup
tmpfs            6446908       16   6446892   1% /run/user/1000


In [50]:
!sudo mkdir /dev/tmp
!sudo chmod a+rwx /dev/tmp
!mv /dev/shm/tmp/permuted-*/ /dev/tmp/

mkdir: cannot create directory ‘/dev/tmp’: File exists


In [52]:
!du -sh /dev/tmp/*

2.0M	/dev/tmp/permuted-0
313M	/dev/tmp/permuted-1
28M	/dev/tmp/permuted-2
141M	/dev/tmp/permuted-3
24G	/dev/tmp/permuted-4
2.7G	/dev/tmp/permuted-5
0	/dev/tmp/tmp


In [9]:

for file in glob('*.tgz'):
    !rm -rf pickles
    run('tar xzf '+file)
    !mv dev/shm/data/tiles/pickles pickles
    !rm -rf dev
    for pickle in glob('pickles/*.pkl'):
        print(pickle)
        sorter.sort_file(pickle)
    break
!ls

/dev/shm/data/unpack


In [60]:
!ls /dev/shm/data

The history saving thread hit an unexpected error (OperationalError('database or disk is full',)).History will not be written to the database.
tiles  unpack  untar


In [50]:
!ls dev/shm/data/tiles/pickles/

tiles_106.aws.pkl  tiles_135.aws.pkl  tiles_45.aws.pkl	tiles_75.aws.pkl
tiles_107.aws.pkl  tiles_136.aws.pkl  tiles_46.aws.pkl	tiles_76.aws.pkl
tiles_108.aws.pkl  tiles_137.aws.pkl  tiles_47.aws.pkl	tiles_77.aws.pkl
tiles_109.aws.pkl  tiles_138.aws.pkl  tiles_48.aws.pkl	tiles_78.aws.pkl
tiles_110.aws.pkl  tiles_139.aws.pkl  tiles_49.aws.pkl	tiles_84.aws.pkl
tiles_111.aws.pkl  tiles_140.aws.pkl  tiles_50.aws.pkl	tiles_85.aws.pkl
tiles_112.aws.pkl  tiles_141.aws.pkl  tiles_51.aws.pkl	tiles_86.aws.pkl
tiles_113.aws.pkl  tiles_150.aws.pkl  tiles_52.aws.pkl	tiles_87.aws.pkl
tiles_114.aws.pkl  tiles_151.aws.pkl  tiles_53.aws.pkl	tiles_88.aws.pkl
tiles_115.aws.pkl  tiles_152.aws.pkl  tiles_54.aws.pkl	tiles_89.aws.pkl
tiles_116.aws.pkl  tiles_153.aws.pkl  tiles_55.aws.pkl	tiles_90.aws.pkl
tiles_117.aws.pkl  tiles_154.aws.pkl  tiles_56.aws.pkl	tiles_91.aws.pkl
tiles_118.aws.pkl  tiles_155.aws.pkl  tiles_64.aws.pkl	tiles_92.aws.pkl
tiles_119.aws.pkl  tiles_156.aws.pkl  tiles_65.aws.pkl	tiles_93.

In [41]:
s3_dir

's3://mousebraindata-open/MD657'

In [17]:
!aws s3 ls $s3_dir/ | grep extracted

2019-02-02 07:11:46          0 MD657-F1-2017.02.17-17.39.26_MD657_1_0001_lossless_extracted.tgz
2019-02-02 06:21:57         45 MD657-F1-2017.02.17-17.39.26_MD657_2_0002_lossless_extracted.tgz
2019-02-02 07:12:21          0 MD657-F1-2017.02.17-17.39.26_MD657_3_0003_lossless_extracted.tgz
2019-02-02 21:37:40         45 MD657-F10-2017.02.17-19.35.42_MD657_1_0028_lossless_extracted.tgz
2019-02-02 18:44:43         45 MD657-F10-2017.02.17-19.35.42_MD657_3_0030_lossless_extracted.tgz
2019-02-02 05:33:55         45 MD657-F10-2017.02.21-17.57.42_MD657_1_0028_lossless_extracted.tgz
2019-02-02 20:48:09         45 MD657-F10-2017.02.21-17.57.42_MD657_2_0029_lossless_extracted.tgz
2019-02-02 07:27:57          0 MD657-F10-2017.02.21-17.57.42_MD657_3_0030_lossless_extracted.tgz
2019-02-02 07:30:50          0 MD657-F11-2017.02.17-19.39.47_MD657_1_0031_lossless_extracted.tgz
2019-02-02 07:21:33          0 MD657-F11-2017.02.17-19.39.47_MD657_2_0032_lossless_extracted.tgz
2019-02-02 07:24:07          0 MD

2019-02-02 19:32:16         45 MD657-F5-2017.02.17-18.29.04_MD657_2_0014_lossless_extracted.tgz
2019-02-02 07:25:59          0 MD657-F5-2017.02.17-18.29.04_MD657_3_0015_lossless_extracted.tgz
2019-02-02 07:30:40          0 MD657-F50-2017.02.18-08.23.22_MD657_1_0148_lossless_extracted.tgz
2019-02-02 07:31:39          0 MD657-F50-2017.02.18-08.23.22_MD657_2_0149_lossless_extracted.tgz
2019-02-02 07:23:03          0 MD657-F51-2017.02.18-08.45.55_MD657_1_0151_lossless_extracted.tgz
2019-02-02 06:50:36         45 MD657-F51-2017.02.18-08.45.55_MD657_2_0152_lossless_extracted.tgz
2019-02-02 20:36:17         45 MD657-F52-2017.02.18-09.06.44_MD657_1_0154_lossless_extracted.tgz
2019-02-02 07:21:23          0 MD657-F52-2017.02.18-09.06.44_MD657_2_0155_lossless_extracted.tgz
2019-02-02 07:19:08          0 MD657-F53-2017.02.18-09.27.00_MD657_1_0157_lossless_extracted.tgz
2019-02-02 20:18:25         45 MD657-F53-2017.02.18-09.27.00_MD657_2_0158_lossless_extracted.tgz
2019-02-02 07:28:45          0 M

2019-02-02 07:16:18          0 MD657-F94-2017.02.18-21.34.40_MD657_3_0282_lossless_extracted.tgz
2019-02-02 07:29:03          0 MD657-F95-2017.02.18-21.46.51_MD657_1_0283_lossless_extracted.tgz
2019-02-02 21:08:19         45 MD657-F95-2017.02.18-21.46.51_MD657_2_0284_lossless_extracted.tgz
2019-02-02 07:20:30          0 MD657-F95-2017.02.18-21.46.51_MD657_3_0285_lossless_extracted.tgz
2019-02-02 07:21:03          0 MD657-F95-2017.02.23-18.13.07_MD657_1_0283_lossless_extracted.tgz
2019-02-02 07:20:02          0 MD657-F95-2017.02.23-18.13.07_MD657_2_0284_lossless_extracted.tgz
2019-02-02 07:10:42          0 MD657-F95-2017.02.23-18.13.07_MD657_3_0285_lossless_extracted.tgz
2019-02-02 21:52:06         45 MD657-F96-2017.02.21-15.43.01_MD657_1_0286_lossless_extracted.tgz
2019-02-02 07:17:15          0 MD657-F96-2017.02.21-15.43.01_MD657_2_0287_lossless_extracted.tgz
2019-02-02 07:24:17          0 MD657-F96-2017.02.21-15.43.01_MD657_3_0288_lossless_extracted.tgz
2019-02-02 07:24:24          0

2019-02-02 07:18:54          0 MD657-N41-2017.02.22-16.12.45_MD657_1_0121_lossless_extracted.tgz
2019-02-02 07:14:42          0 MD657-N41-2017.02.22-16.12.45_MD657_2_0122_lossless_extracted.tgz
2019-02-02 07:24:14          0 MD657-N42-2017.02.22-16.16.50_MD657_1_0124_lossless_extracted.tgz
2019-02-02 07:12:27          0 MD657-N42-2017.02.22-16.16.50_MD657_2_0125_lossless_extracted.tgz
2019-02-02 07:11:04          0 MD657-N43-2017.02.22-16.21.20_MD657_1_0127_lossless_extracted.tgz
2019-02-02 07:15:33          0 MD657-N43-2017.02.22-16.21.20_MD657_2_0128_lossless_extracted.tgz
2019-02-02 07:22:59          0 MD657-N44-2017.02.22-16.25.25_MD657_1_0130_lossless_extracted.tgz
2019-02-02 19:28:11  969451902 MD657-N44-2017.02.22-16.25.25_MD657_2_0131_lossless_extracted.tgz
2019-02-02 07:25:35          0 MD657-N45-2017.02.22-16.29.38_MD657_1_0133_lossless_extracted.tgz
2019-02-02 07:24:10          0 MD657-N45-2017.02.22-16.29.38_MD657_2_0134_lossless_extracted.tgz
2019-02-02 07:21:36          0

2019-02-02 07:23:49          0 MD657-N84-2017.02.22-19.16.42_MD657_1_0250_lossless_extracted.tgz
2019-02-02 07:18:02          0 MD657-N84-2017.02.22-19.16.42_MD657_2_0251_lossless_extracted.tgz
2019-02-02 07:26:47          0 MD657-N84-2017.02.22-19.16.42_MD657_3_0252_lossless_extracted.tgz
2019-02-02 05:36:28  438918438 MD657-N85-2017.02.22-19.20.56_MD657_1_0253_lossless_extracted.tgz
2019-02-02 07:09:26          0 MD657-N85-2017.02.22-19.20.56_MD657_2_0254_lossless_extracted.tgz
2019-02-02 07:13:50          0 MD657-N85-2017.02.22-19.20.56_MD657_3_0255_lossless_extracted.tgz
2019-02-02 06:29:11  443410310 MD657-N86-2017.02.22-19.25.03_MD657_1_0256_lossless_extracted.tgz
2019-02-02 07:15:23          0 MD657-N86-2017.02.22-19.25.03_MD657_2_0257_lossless_extracted.tgz
2019-02-02 07:18:34          0 MD657-N86-2017.02.22-19.25.03_MD657_3_0258_lossless_extracted.tgz
2019-02-02 07:30:12          0 MD657-N87-2017.02.22-19.29.10_MD657_1_0259_lossless_extracted.tgz
2019-02-02 19:47:37  379180283

In [34]:
from  lib.utils import list_s3_files,run
s3_dir='s3://mousebraindata-open/MD657'

L=list_s3_files(s3_dir)

runPipe cmd= aws s3 ls s3://mousebraindata-open/MD657/ 


In [35]:
from os import stat,remove
i=0
for file in L:
    if 'extract' in file:
        print(file)
        path='/dev/shm/data/patches/'+file
        run('aws s3 cp {0}/{1} /dev/shm/data/patches/{1}'.format(s3_dir,file))
        file_stat=stat(path)
        print(file_stat.st_size)
        if(file_stat.st_size<1000):
            remove(path)
        else:
            i+=1
            if(i>10):
                break

MD657-F73-2017.02.18-15.46.36_MD657_1_0217_lossless_extracted.tgz
cmd= aws s3 cp s3://mousebraindata-open/MD657/MD657-F73-2017.02.18-15.46.36_MD657_1_0217_lossless_extracted.tgz /dev/shm/data/patches/MD657-F73-2017.02.18-15.46.36_MD657_1_0217_lossless_extracted.tgz
45
MD657-F78-2017.02.18-17.13.40_MD657_1_0232_lossless_extracted.tgz
cmd= aws s3 cp s3://mousebraindata-open/MD657/MD657-F78-2017.02.18-17.13.40_MD657_1_0232_lossless_extracted.tgz /dev/shm/data/patches/MD657-F78-2017.02.18-17.13.40_MD657_1_0232_lossless_extracted.tgz
45
MD657-F87-2017.02.18-19.44.12_MD657_1_0259_lossless_extracted.tgz
cmd= aws s3 cp s3://mousebraindata-open/MD657/MD657-F87-2017.02.18-19.44.12_MD657_1_0259_lossless_extracted.tgz /dev/shm/data/patches/MD657-F87-2017.02.18-19.44.12_MD657_1_0259_lossless_extracted.tgz
45
MD657-N11-2017.02.22-14.29.36_MD657_1_0031_lossless_extracted.tgz
cmd= aws s3 cp s3://mousebraindata-open/MD657/MD657-N11-2017.02.22-14.29.36_MD657_1_0031_lossless_extracted.tgz /dev/shm/data/p

In [36]:
!ls -l /dev/shm/data/patches/

total 5380392
-rw-rw-r-- 1 ubuntu ubuntu 459643052 Feb  2 19:23 MD657-N11-2017.02.22-14.29.36_MD657_1_0031_lossless_extracted.tgz
-rw-rw-r-- 1 ubuntu ubuntu 469935563 Feb  2 20:56 MD657-N11-2017.02.22-14.29.36_MD657_2_0032_lossless_extracted.tgz
-rw-rw-r-- 1 ubuntu ubuntu 452962329 Feb  2 05:50 MD657-N11-2017.02.22-14.29.36_MD657_3_0033_lossless_extracted.tgz
-rw-rw-r-- 1 ubuntu ubuntu 441391844 Feb  2 20:05 MD657-N12-2017.02.27-16.04.52_MD657_3_0036_lossless_extracted.tgz
-rw-rw-r-- 1 ubuntu ubuntu 547587040 Feb  2 21:55 MD657-N15-2017.02.22-14.41.51_MD657_2_0044_lossless_extracted.tgz
-rw-rw-r-- 1 ubuntu ubuntu 544733701 Feb  2 06:32 MD657-N15-2017.02.22-14.41.51_MD657_3_0045_lossless_extracted.tgz
-rw-rw-r-- 1 ubuntu ubuntu 608248061 Feb  2 05:47 MD657-N17-2017.02.22-14.45.59_MD657_1_0049_lossless_extracted.tgz
-rw-rw-r-- 1 ubuntu ubuntu 589587251 Feb  2 05:53 MD657-N17-2017.02.22-14.45.59_MD657_3_0051_lossless_extracted.tgz
-rw-rw-r-- 1 ubuntu ubuntu 643530078 Feb  2 20:11 MD657-N1

In [20]:
from lib.utils import get_file_table

ImportError: cannot import name 'get_file_table'